In [1]:
import os
import sys
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyts.image import GramianAngularField

In [2]:
if sys.platform == "win32":
    dataPath = ""
elif sys.platform == "darwin":
    dataPath = "/Users/jwh/Desktop/0919/Test2/result/"

fileNames = [file for file in sorted(glob.glob(dataPath + "*.csv")) if os.path.isfile(file)]

In [3]:
sensorNames = [f"s{i}" for i in np.arange(1, 7, 1)]

numRows = 4000

In [4]:
fileIdx = [31, 32, 33,                  # class 1
           58, 59, 60,                  # class 2
           61, 62, 63,                  # class 3
           64, 65, 66,                  # class 4
           67, 68, 69,                  # class 5
           70, 71, 72,                  # class 6
           73, 74, 75,                  # class 7
           76, 77, 78,                  # class 8
           79, 80, 81,                  # class 9
           1, 2, 3,                     # class 10
           4, 5, 6,                     # class 11
           7, 8, 9,                     # class 12
           10, 11, 12,                  # class 13
           13, 14, 15,                  # class 14
           16, 17, 18,                  # class 15
           19, 20, 21,                  # class 16
           22, 23, 24,                  # class 17
           25, 26, 27,                  # class 18
           28, 29, 30,                  # class 19
           34, 35, 36,                  # class 20
           37, 38, 39,                  # class 21
           40, 41, 42,                  # class 22
           43, 44, 45,                  # class 23
           46, 47, 48,                  # class 24
           49, 50, 51,                  # class 25
           52, 53, 54,                  # class 26
           55, 56, 57]                  # class 27

sortedFiles = [fileNames[i] for i in fileIdx]

In [5]:
files = [pd.read_csv(file, sep= ",", nrows= numRows) for file in sortedFiles]

In [6]:
base = pd.read_csv(fileNames[0], sep= ",", nrows= numRows)
meanBase = pd.DataFrame(base.mean(axis= 0), columns= ['base'])
meanBase.head()

,base
s1,-828.26300
s2,-854.03200
s3,-886.97450
s4,756.29175
s5,744.35150


In [7]:
def calibration(df):
    res = [df.iloc[:, i] - meanBase.iloc[i, 0] for i in range(6)]
    res = np.array(res).reshape(6, -1).T
    res = pd.DataFrame(res, columns= sensorNames)
    return res

In [8]:
calibrated = [calibration(files[i]) for i in range(len(files))]
calibrated = np.array(calibrated)
calibrated = calibrated.reshape(81, 6, 4000, 1)
print(calibrated.shape)

(81, 6, 4000, 1)


In [9]:
splitedCase = []
for i in range(calibrated.shape[0]):
    splitedSensor = []
    for j in range(calibrated.shape[1]):
        splitedData = []
        for k in range(int(len(calibrated[i, j]) / 16)):
            splited = calibrated[i, j, k * 16 : (16 + (k * 16)), :]
            splitedData.append(splited)
        splitedData = np.array(splitedData)
        splitedSensor.append(splitedData)
    splitedSensor = np.array(splitedSensor)
    splitedCase.append(splitedSensor)
splitedCase = np.array(splitedCase)

print(splitedCase.shape)

(81, 6, 250, 16, 1)


In [10]:
gaf = GramianAngularField(method= 'difference')

In [11]:
encodedResult = []
for i in range(splitedCase.shape[0]):
    encodedSensor = []
    for j in range(splitedCase.shape[1]):
        encodedData = []
        for k in range(splitedCase.shape[2]):
            data = splitedCase[i, j, k, :]
            encoded = gaf.fit_transform(data.T)
            encoded = encoded.reshape(16, 16)
            encodedData.append(encoded)
        encodedData = np.array(encodedData)
        encodedSensor.append(encodedData)
    encodedSensor = np.array(encodedSensor)
    encodedResult.append(encodedSensor)
encodedResult = np.array(encodedResult)

print(encodedResult.shape)

(81, 6, 250, 16, 16)


In [12]:
concatenateResult = []
for c in range(encodedResult.shape[0]):
    sensor1 = encodedResult[c, 0]
    sensor2 = encodedResult[c, 1]
    sensor3 = encodedResult[c, 2]
    sensor4 = encodedResult[c, 3]
    sensor5 = encodedResult[c, 4]
    sensor6 = encodedResult[c, 5]

    concatenateImg = []
    for d in range(encodedResult.shape[2]):
        sensor1Img = sensor1[d].reshape(16, 16, 1)
        sensor2Img = sensor2[d].reshape(16, 16, 1)
        sensor3Img = sensor3[d].reshape(16, 16, 1)
        sensor4Img = sensor4[d].reshape(16, 16, 1)
        sensor5Img = sensor5[d].reshape(16, 16, 1)
        sensor6Img = sensor6[d].reshape(16, 16, 1)

        res = np.concatenate((sensor1Img, sensor2Img), axis= 2)
        res = np.concatenate((res, sensor3Img), axis= 2)
        res = np.concatenate((res, sensor4Img), axis= 2)
        res = np.concatenate((res, sensor5Img), axis= 2)
        res = np.concatenate((res, sensor6Img), axis= 2)

        concatenateImg.append(res)

    concatenateImg = np.array(concatenateImg)
    concatenateResult.append(concatenateImg)
concatenateResult = np.array(concatenateResult)

print(concatenateResult.shape)

(81, 250, 16, 16, 6)


In [13]:
idxList = [i for i in np.arange(0, 81, 1)]
classIdx = []
for i in range(27):
    tmp = idxList[i*3 : (3 + (i * 3))]
    classIdx.append(tmp)

classIdx = np.array(classIdx)

In [14]:
concateClass = []
for i in range(classIdx.shape[0]):
    c = [concatenateResult[idx] for idx in classIdx[i]]
    concateClass.append(c)

concateClass = np.array(concateClass)

# for c in range(concateClass.shape[0]):
#     print(f"Class {c}: {concateClass[c].shape}")

In [16]:
def makeTrainingData(concateClass, label, classNum):
    counter = 0

    classImg = []
    classLabelList = []
    yClass = []

    for n in range(concateClass.shape[0]):
        for j in range(concateClass.shape[1]):
            img1 = concateClass[n, d, :, :, :3]
            img2 = concateClass[n, d, :, :, 3:6]

            classImg.append(img1)
            classLabelList.append(f"{label}_{counter}")
            yClass.append(classNum)

            counter += 1

            classImg.append(img2)
            classLabelList.append(f"{label}_{counter}")
            yClass.append(classNum)

            counter += 1

    classImg = np.array(classImg)
    classLabelList = np.array(classLabelList)
    yClass = np.array(yClass)

    return classImg, classLabelList, yClass

In [19]:
a, b, c, = makeTrainingData(concateClass[0], label= "C1", classNum= 0)

In [20]:
a.shape

(1500, 16, 16, 3)

In [37]:
X_data = []
i_data = []
y_data = []
for i in range(concateClass.shape[0]):
    classImg, classLabel, yClass = makeTrainingData(concateClass= concateClass[i], label= f"C{i}", classNum= i)
    X_data.append(classImg)
    i_data.append(classLabel)
    y_data.append(yClass)

X_data = np.concatenate(X_data)
i_data = np.concatenate(i_data)
y_data = np.concatenate(y_data)

X_data = np.array(X_data)
y_data = np.array(y_data)
i_data = np.array(i_data)

print(X_data.shape)
print(i_data.shape)
print(y_data.shape)

(40500, 16, 16, 3)
(40500,)
(40500,)


In [30]:
tmp = [[x, y, z] for x, y, z in zip(X_data, y_data, i_data)]

random.shuffle(tmp)

X_data = [n[0] for n in tmp]
y_data = [n[1] for n in tmp]
i_data = [n[2] for n in tmp]

# npzDataPath = "./"
# np.savez(os.path.join(npzDataPath, "Test0919"), X_data= X_data, y_data= y_data, i_data= i_data)